In [1]:
import json
import os
import sys
import getpass
import hashlib
import uuid
from datetime import datetime
from typing import Dict, Any, List

In [2]:
DATA_FILE = "lms_data.json"

In [ ]:
# Utilities
def now_str():
    return datetime.now().isoformat(timespec="seconds")


def hash_password(password: str) -> str:
    return hashlib.sha256(password.encode("utf-8")).hexdigest()


def clear_screen():
    os.system("cls" if os.name == "nt" else "clear")


def pause(msg="Press Enter to continue..."):
    input(msg)

In [ ]:
# Data model helpers

def load_data():
    if not os.path.exists(DATA_FILE):
        return {"users": {}, "courses": {}}
    with open(DATA_FILE, "r", encoding="utf-8") as f:
        return json.load(f)


def save_data(data):
    with open(DATA_FILE, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=2)


In [ ]:
# User and Auth
def create_user(data, username, password, role, full_name=""):
    if username in data["users"]:
        return False, "username already exists"
    uid = str(uuid.uuid4())
    data["users"][username] = {
        "id": uid,
        "username": username,
        "full_name": full_name,
        "password_hash": hash_password(password),
        "role": role,
        "enrolled": [],         # list of course_ids for students
        "created_courses": [],  # list of course_ids for teachers
        "grades": {}            # {course_id: {quiz_id: score}}
    }
    save_data(data)
    return True, "user created"


def authenticate(data, username, password):
    user = data["users"].get(username)
    if not user:
        return None
    if user["password_hash"] == hash_password(password):
        return user
    return None
